In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.misc import toimage
from keras.datasets import mnist
from keras.utils import np_utils

In [ ]:
from keras.datasets import mnist

## KerasのサンプルデータセットのMNISTのデータ読み込み

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train.shape

In [ ]:
X_train[1]

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
y_train[3]

In [ ]:
Y_train = np_utils.to_categorical(y_train, 10)

In [ ]:
Y_train[3]

In [ ]:
Y_test = np_utils.to_categorical(y_test, 10)

## Kerasでパーセプトロンの実装とMNISTのテスト

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam
from keras.utils.visualize_util import plot

In [ ]:
def plot_history(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.legend(['acc', 'val_acc'], loc='lower right')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.legend(['loss', 'val_loss'], loc='lower right')
    plt.show()

In [ ]:
def test_mnist():
    test_num = 10
    start = np.random.randint(X_test.shape[0] - test_num)
    x_test = X_test[start:start+test_num]
    y_test = Y_test[start:start+test_num]

    for i in range(x_test.shape[0]):
        img = toimage(x_test[i])
        plt.subplot(1, test_num, i + 1)
        plt.axis('off')
        plt.imshow(img)
    plt.show()

    print(np.argmax(y_test, axis=1))

    preds = model.predict(x_test.reshape(test_num,784))
    print(np.argmax(preds, axis=1))

In [ ]:
model = Sequential()
model.add(Dense(10, input_shape=(784,)))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='SGD',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train.reshape(60000, 784), Y_train,
                    validation_data=(X_test.reshape(10000, 784), Y_test),
                    batch_size=30, nb_epoch=5)

In [ ]:
plot_history(history)

In [ ]:
test_mnist()

## Kerasでバックプロパゲーションの実装とMNISTのテスト

In [ ]:
model = Sequential()
model.add(Dense(200, input_dim=784)) # 28x28
model.add(Activation("relu"))
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='SGD',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train.reshape(60000, 784), Y_train,
                    validation_data=(X_test.reshape(10000, 784), Y_test),
                    batch_size=30, nb_epoch=3)

In [ ]:
plot_history(history)

In [ ]:
test_mnist()

In [ ]:
model = Sequential()
model.add(Dense(200, input_dim=784)) # 28x28
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(10))
model.add(Activation("sigmoid"))

In [ ]:
model.compile(optimizer='SGD',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train.reshape(60000, 784), Y_train,
                    validation_data=(X_test.reshape(10000, 784), Y_test),
                    batch_size=30, nb_epoch=3)

In [ ]:
plot_history(history)

In [ ]:
test_mnist()

In [ ]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train.reshape(60000, 784), Y_train,
                    validation_data=(X_test.reshape(10000, 784), Y_test),
                    batch_size=30, nb_epoch=3)

In [ ]:
plot_history(history)

In [ ]:
test_mnist()

## Kerasで小さなCNNの実装とMNISTのテスト

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

In [ ]:
model = Sequential()
model.add(Convolution2D(20, 3, 3, border_mode='same', input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(40, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(50))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
# 5分くらいかかるので注意
history = model.fit(X_train.reshape(60000, 28, 28, 1), Y_train,
                    validation_data=(X_test.reshape(10000, 28, 28, 1), Y_test),
                    batch_size=30, nb_epoch=1)

In [ ]:
plot_history(history)

In [ ]:
test_mnist()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# http://aidiary.hatenablog.com/entry/20161120/1479640534
def visualize_filter(model):
    W = model.layers[0].get_weights()[0]
    # (nb_filter, nb_channel, nb_row, nb_col)
    W = W.transpose(3, 2, 0, 1)
    (nb_filter, nb_channel, nb_row, nb_col) = W.shape

    plt.figure()
    for i in range(nb_filter):
        im = W[i, 0]
        scaler = MinMaxScaler(feature_range=(0, 255))
        im = scaler.fit_transform(im)

        plt.subplot(4, 8, i + 1)
        plt.axis('off')
        plt.imshow(im, cmap="gray")
    plt.show()

In [ ]:
visualize_filter(model)

## Kerasで小さなCNNでCIFAR10のテスト

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

In [ ]:
model = Sequential()
model.add(Convolution2D(40, 3, 3, border_mode='same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(60, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(50))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

In [ ]:
# 10分くらいかかるので注意
history = model.fit(X_train, Y_train,
                    validation_data=(X_test, Y_test),
                    batch_size=30, nb_epoch=1)

In [ ]:
def test_cifar10():
    test_num = 10
    start = np.random.randint(X_test.shape[0] - test_num)
    x_test = X_test[start:start+test_num]
    y_test = Y_test[start:start+test_num]

    for i in range(x_test.shape[0]):
        img = toimage(x_test[i])
        plt.subplot(1, test_num, i + 1)
        plt.axis('off')
        plt.imshow(img)
    plt.show()

    print(np.argmax(y_test, axis=1))

    preds = model.predict(x_test)
    print(np.argmax(preds, axis=1))

In [ ]:
# 0:airplane, 1:automobile, 2:bird, 3:cat, 4:deer, 5:dog, 6:frog, 7:horse, 8:ship, 9:truck

In [ ]:
test_cifar10()

In [ ]:
def visualize_filter_color(model):
    W = model.layers[0].get_weights()[0]
    # (nb_filter, nb_channel, nb_row, nb_col)
    W = W.transpose(3, 2, 0, 1)
    (nb_filter, nb_channel, nb_row, nb_col) = W.shape

    plt.figure()
    rgb_hash = {0: "Reds", 1: "Greens", 2: "Blues"}
    for rgb in range(3):
        for i in range(nb_filter):
            im = W[i, rgb]
            scaler = MinMaxScaler(feature_range=(0, 255))
            im = scaler.fit_transform(im)

            plt.subplot(40*3//10, 10, rgb*40 + i + 1)
            plt.axis('off')
            plt.imshow(im, cmap=rgb_hash[rgb])
    plt.show()

In [ ]:
visualize_filter_color(model)

## 重みデータのsave/load

In [ ]:
#model.save_weights("cifar10_model_weights.h5")
model_weights = model.get_weights()
np.save("cifar10_model_weights.npy", model_weights)

In [ ]:
#model.load_weights("model_weights.h5")
model_weights = np.load("cifar10_model_weights.npy")
model.set_weights(model_weights)